In [3]:
import os

path = 'lastfm-dataset-1K'

if not os.path.exists(path):

  !wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
  !tar -xvzf lastfm-dataset-1K.tar.gz


In [1]:
!ls

drive  lastfm-dataset-1K  lastfm-dataset-1K.tar.gz  sample_data


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls drive/MyDrive/data

feature_df_with_emb.csv  spotify_features.csv


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import scipy.sparse as sp
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import datetime
import time 
import math
import os
%config Completer.use_jedi = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.


In [6]:
path = "drive/MyDrive/data/feature_df_with_emb.csv"

feature_df_with_emb = pd.read_csv(path)

In [7]:
feature_df_with_emb.head()

,Unnamed: 0,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
0,0,坂本龍一,Composition 0919 (Live_2009_4_15),0.806,0.413,5.0,-12.883,1.0,0.1560,0.987,0.794,0.1130,0.5290,105.089,330253.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
1,1,坂本龍一,Hibari (Live_2009_4_15),0.604,0.283,9.0,-19.811,0.0,0.0583,0.996,0.916,0.1140,0.2700,137.802,540133.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
2,2,坂本龍一,To Stanford (Live_2009_4_15),0.453,0.163,5.0,-20.922,1.0,0.0519,0.990,0.939,0.1010,0.0703,73.926,216853.0,"['japanese indie folk', 'japanese soundtrack',...",39,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
3,3,坂本龍一,Improvisation (Live_2009_4_15),0.217,0.012,5.0,-36.464,1.0,0.0376,0.991,0.859,0.0866,0.0376,80.714,384707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
4,4,坂本龍一,Glacier (Live_2009_4_15),0.518,0.213,9.0,-28.790,0.0,0.2090,0.869,0.898,0.1030,0.0920,104.191,580707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493


In [8]:
# Load the dataset
df = pd.read_table('lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', lineterminator='\n', warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
df_profile = pd.read_csv('lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=True, skiprows=1, names=['user', 'gender', 'age', 'country', 'signup'])

In [9]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [10]:
# Get rows with song names containing multiple rows
duplicated = df.loc[df['song-id'].isin(grouped[grouped['song'] > 1].index)]
duplicated = duplicated.drop(columns=['user', 'timestamp', 'artist', 'artist-id'])
containis_extra_rows = duplicated.apply(lambda x: pd.Series({'id': x[0], 'song': x[1], 'flag':'\n' in x[1]}), axis=1)
containis_extra_rows = containis_extra_rows.loc[containis_extra_rows['flag']]

In [11]:
# Iterate over those 5k rows to get the extra rows and correct the song name
for index, row in containis_extra_rows.iterrows():
    row_break = row['song'].split('\n', 1)
    song_name = row_break[0]
    df.loc[df['song-id'] == row['id'], 'song'] = song_name
    
    tsv = StringIO(row_break[1])
    df_extra = pd.read_csv(tsv, sep="\t", warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
    df = df.append(df_extra, ignore_index=True)

### Fix names

In [12]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [13]:
# Check how many ids has more than 1 song name
grouped[grouped['song'] > 1].shape[0]

200

In [14]:
ids_to_be_fixed = grouped[grouped['song'] > 1].index

In [15]:
df = df.set_index('song-id')

In [16]:
map_to_fix = df.groupby(df.index)['song'].head(1)

In [17]:
df.loc[ids_to_be_fixed, 'song'] = df.loc[ids_to_be_fixed].reset_index()['song-id'].map(lambda x: map_to_fix[x])
df = df.reset_index()

### Create matrix

In [18]:
matrix_init = df.groupby(['user', 'artist', 'song']).count()

In [19]:
matrix_init = matrix_init['timestamp']

In [20]:
matrix_init = matrix_init.reset_index()

In [21]:
def rating_scaler(row):
    row_array = np.array(row)
    a, new_range= 1, 4
    min_, max_ = row_array.min(), row_array.max()
    old_range = max_ - min_
    
    
    scaled_row = (new_range * (row_array - min_)) / (old_range + 1e-6)  + a
    return pd.Series(scaled_row)

In [22]:
matrix_init_scaled = matrix_init.groupby('user')['timestamp'].apply(rating_scaler)

In [23]:
matrix_init['timestamp'] = matrix_init_scaled.reset_index()['timestamp']

In [24]:
feature_df_with_emb.head()

,Unnamed: 0,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
0,0,坂本龍一,Composition 0919 (Live_2009_4_15),0.806,0.413,5.0,-12.883,1.0,0.1560,0.987,0.794,0.1130,0.5290,105.089,330253.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
1,1,坂本龍一,Hibari (Live_2009_4_15),0.604,0.283,9.0,-19.811,0.0,0.0583,0.996,0.916,0.1140,0.2700,137.802,540133.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
2,2,坂本龍一,To Stanford (Live_2009_4_15),0.453,0.163,5.0,-20.922,1.0,0.0519,0.990,0.939,0.1010,0.0703,73.926,216853.0,"['japanese indie folk', 'japanese soundtrack',...",39,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
3,3,坂本龍一,Improvisation (Live_2009_4_15),0.217,0.012,5.0,-36.464,1.0,0.0376,0.991,0.859,0.0866,0.0376,80.714,384707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
4,4,坂本龍一,Glacier (Live_2009_4_15),0.518,0.213,9.0,-28.790,0.0,0.2090,0.869,0.898,0.1030,0.0920,104.191,580707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493


### Merge data

In [25]:
matrix_init = matrix_init.rename({'timestamp':'rating', 'song':'track'}, axis=1)

In [26]:
matrix_extended = matrix_init.merge(feature_df_with_emb, on=['artist', 'track'])

In [27]:
matrix_extended.head()

,user,artist,track,rating,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
0,user_000001,2562,Basin Dub,1.057971,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
1,user_000451,2562,Basin Dub,1.045455,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
2,user_000471,2562,Basin Dub,1.000000,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
3,user_000508,2562,Basin Dub,1.166667,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
4,user_000515,2562,Basin Dub,1.027586,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273


### Filter the dataset

In [ ]:
matrix_extended['item'] = matrix_extended.apply(lambda row: row[1]+'-'+row[2], axis=1)

In [218]:
matrix_extended.groupby('user').count()['rating'].median()

2945.0

### Sample Users

In [225]:
no_users = 200
users_list = matrix_extended.user.unique()
users_sampled = np.random.choice(users_list, replace=False, size=no_users)
matrix_sampled = matrix_extended.copy()
matrix_sampled = matrix_sampled.set_index('user')
matrix_sampled = matrix_sampled.loc[users_sampled]
matrix_sampled = matrix_sampled.reset_index()
matrix_sampled = matrix_sampled.sample(frac=1)
matrix_sampled['item'] = matrix_sampled.apply(lambda row: row[1]+'-'+row[2], axis=1)

In [226]:
matrix_sampled.head()

,user,artist,track,rating,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z,item
933560,user_000875,Curtis Fuller,Minor Vamp,1.000000,1035028,0.578,0.250,1.0,-19.047,1.0,0.0544,0.8660,0.086300,0.0874,0.702,124.861,313627.0,['hard bop'],30,"(3.70538, 5.647046, 0.40997434)",3.705380,5.647046,0.409974,Curtis Fuller-Minor Vamp
649532,user_000437,Manu Chao,Me Gustas Tu,1.027972,23285,0.657,0.741,11.0,-6.822,0.0,0.0825,0.6630,0.494000,0.2220,0.908,175.949,239987.0,"['world', 'latin alternative']",71,[0.93859445 6.3103405 2.30777515],0.938594,6.310341,2.307775,Manu Chao-Me Gustas Tu
616620,user_000572,The Undertones,My Perfect Cousin,1.800000,45453,0.509,0.858,9.0,-10.257,1.0,0.0508,0.0038,0.002580,0.0762,0.382,158.627,156360.0,"['punk', 'new wave pop', 'new wave', 'northern...",52,[5.66876385 8.4624205 0.53306184],5.668764,8.462421,0.533062,The Undertones-My Perfect Cousin
901155,user_000702,Gilbert Bécaud,Nathalie,1.000000,484045,0.422,0.632,9.0,-7.121,0.0,0.0779,0.7490,0.000001,0.5940,0.810,134.715,245213.0,['chanson'],54,"(3.4330428, 6.9000406, 2.3754716)",3.433043,6.900041,2.375472,Gilbert Bécaud-Nathalie
429332,user_000451,Stevie Wonder,That Girl,1.000000,314618,0.792,0.691,6.0,-8.975,1.0,0.0431,0.1020,0.000017,0.0679,0.899,106.979,313840.0,"['indie r&b', 'soul', 'motown', 'adult standar...",80,[3.7881843 6.35666815 1.04414575],3.788184,6.356668,1.044146,Stevie Wonder-That Girl


In [227]:
matrix_sampled['user_id'] = matrix_sampled['user'].astype('category').cat.codes
matrix_sampled['item_id'] = matrix_sampled['item'].astype('category').cat.codes

In [228]:
X = matrix_sampled[['user_id', 
              'item_id',
              'danceability',
              'energy',
              'speechiness', 
              'acousticness', 
              'instrumentalness', 
              'liveness',
              'valence', 
              'tempo', 
              'popularity', 
              'emb_x', 
              'emb_y',
              'emb_z']]
y = matrix_sampled['rating']
groups = matrix_sampled['user_id']

In [229]:
# Change splitting to be by index using https://stackoverflow.com/questions/53490497/getting-validation-set-from-train-set-by-using-percentage-from-groupby-in-pand
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=groups)

In [230]:
no_users, no_items, no_factors = matrix_sampled['user_id'].nunique(), matrix_sampled['item_id'].nunique(), 100

### Models

In [231]:
def create_shallow_model(no_factors, no_users, no_items):
    # User branch
    user_id = tf.keras.layers.Input(shape=[1], name='user_id')
    user_matrix = tf.keras.layers.Embedding(no_users+1, no_factors, name='user_matrix')(user_id)
    user_vector = tf.keras.layers.Flatten(name='user_vector')(user_matrix)
    # Item branch
    item_id = tf.keras.layers.Input(shape=[1], name='item_id')
    item_matrix = tf.keras.layers.Embedding(no_items+1, no_factors, name='item_matrix')(item_id)
    item_vector = tf.keras.layers.Flatten(name='item_vector')(item_matrix)
    # Dot product 
    vectors_product = tf.keras.layers.dot([user_vector, item_vector], axes=1, normalize=False)
    # Model definition
    model = tf.keras.models.Model(inputs=[user_id, item_id], outputs=[vectors_product], name='shallow_model')
    return model

In [232]:
model = create_shallow_model(no_factors, no_users, no_items)

In [233]:
model.summary()

Model: "shallow_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_matrix (Embedding)         (None, 1, 100)       20100       user_id[0][0]                    
__________________________________________________________________________________________________
item_matrix (Embedding)         (None, 1, 100)       38512500    item_id[0][0]                    
______________________________________________________________________________________

In [234]:
def create_deep_model(no_factors, no_users, no_items):
    # User branch
    user_id = tf.keras.layers.Input(shape=[1], name='user_id')
    user_matrix = tf.keras.layers.Embedding(no_users+1, no_factors, name='user_matrix')(user_id)
    user_vector = tf.keras.layers.Flatten(name='user_vector')(user_matrix)
    # Item branch
    item_id = tf.keras.layers.Input(shape=[1], name='item_id')
    item_matrix = tf.keras.layers.Embedding(no_items+1, no_factors, name='item_matrix')(item_id)
    item_vector = tf.keras.layers.Flatten(name='item_vector')(item_matrix)
    # Concantenation
    vectors_concat = tf.keras.layers.Concatenate()([user_vector, item_vector])
    vectors_concat_dropout = tf.keras.layers.Dropout(0.2)(vectors_concat)
    # Backbone 
    dense_1 = tf.keras.layers.Dense(128,name='fc3')(vectors_concat_dropout)
    dropout_1 = tf.keras.layers.Dropout(0.2,name='d3')(dense_1)
    dense_2 = tf.keras.layers.Dense(8,name='fc4', activation='relu')(dropout_1)
    dense_2_output = tf.keras.layers.Dense(1, activation='relu', name='activation')(dense_2)
    # Model definition
    model = tf.keras.models.Model(inputs=[user_id, item_id], outputs=[dense_2_output], name='deep_model')
    return model

In [235]:
model = create_deep_model(no_factors, no_users, no_items)

### Model training

In [236]:
# Input-output data definition
X_train_arr = [X_train.user_id, X_train.item_id]

# Model creation
model = create_deep_model(no_factors, no_users, no_items)

# Model compiling 
model.compile(loss=tf.keras.losses.MeanSquaredError())

# Model training
model.fit(X_train_arr, y_train, epochs=10, batch_size=2048, shuffle=True)

Epoch 1/10
379/379 [==============================] - 63s 165ms/step - loss: 0.1018
Epoch 2/10
379/379 [==============================] - 63s 165ms/step - loss: 0.0854
Epoch 3/10
379/379 [==============================] - 62s 165ms/step - loss: 0.0806
Epoch 4/10
379/379 [==============================] - 62s 164ms/step - loss: 0.0734
Epoch 5/10
379/379 [==============================] - 62s 164ms/step - loss: 0.0674
Epoch 6/10
379/379 [==============================] - 62s 164ms/step - loss: 0.0631
Epoch 7/10
379/379 [==============================] - 62s 165ms/step - loss: 0.0602
Epoch 8/10
379/379 [==============================] - 62s 164ms/step - loss: 0.0580
Epoch 9/10
379/379 [==============================] - 62s 164ms/step - loss: 0.0562
Epoch 10/10
379/379 [==============================] - 62s 164ms/step - loss: 0.0546


### Prediction

In [237]:
# Predictions in the training set
# X_train = [X_train.user_id, X_train.item_id]
y_train_pred = model.predict(X_train_arr, batch_size=2048)

In [238]:
X_test.head()

,user_id,item_id,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,emb_x,emb_y,emb_z
147544,168,328353,0.462,0.877,0.0414,0.15700,0.000010,0.117,0.885,129.726,50,7.098665,7.336328,2.307446
319554,32,36140,0.463,0.673,0.9400,0.71600,0.000000,0.960,0.302,76.702,34,7.134524,8.881257,0.943933
782879,84,208692,0.638,0.955,0.0274,0.00786,0.000128,0.160,0.600,137.987,62,6.133835,6.965524,3.720656
216677,35,153616,0.749,0.583,0.0942,0.20900,0.003200,0.103,0.897,132.658,46,2.815995,7.732660,2.081691
843012,26,82378,0.733,0.670,0.1320,0.11900,0.000000,0.442,0.763,86.983,65,4.237202,7.522562,4.379738


In [239]:
# Predictions in the test set
X_test_arr = [X_test.user_id, X_test.item_id]
y_test_pred = model.predict(X_test_arr, batch_size=2048)

### Evaluation

In [240]:
from sklearn.metrics import mean_squared_error
print('Train RMSE:', mean_squared_error(y_train.values, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test.values, y_test_pred, squared=False))

Train RMSE: 0.21970625318146253
Test RMSE: 0.32491139547904274


In [242]:
def precision_at_k(model, pred_func, train_ratings, test_ratings, no_users, no_items, k=5):
    precisions = []
    # For each user
    for user_id, user_test_rating in tqdm(test_ratings.groupby('user_id')):
        # Retrieve already-seen items
        train_pids = train_ratings[train_ratings['user_id'] == user_id]['item_id'].values
        # Retrieve the unseen items
        user_df = test_ratings[(test_ratings['user_id'] == user_id)]
        test_pids = user_df['item_id'].values
        relevant_pids = set(user_df.loc[user_df['rating'] > 2.5, 'item_id'].values)
        if len(relevant_pids) == 0:
          continue
        # Create testing dataframe
        test_df = pd.DataFrame({'user_id': user_id, 'item_id': test_pids})
        test_set = [test_df.user_id, test_df.item_id]
        # Make rating predictions for all items for that user
        predictions = model.predict(test_set, batch_size=2048)
        # predictions = pred_func(model, user_id, pid_array, train_ratings)
        predictions = predictions.reshape(-1)
        # Sort the items and het the top k
        top_k = np.argsort(predictions)[:k]
        top_k_items = set(np.array(test_pids)[top_k[:10]])
        # Compute precision as per definition
        precisions.append(len(top_k_items & relevant_pids) / float(k))
    return precisions

In [243]:
X_train_arr = np.array(X_train_arr)
X_test_arr = np.array(X_test_arr)

In [244]:
train_ratings = pd.DataFrame({'user_id': X_train_arr[0, :], 'item_id':X_train_arr[1, :], 'rating': y_train})
test_ratings = pd.DataFrame({'user_id': X_test_arr[0, :], 'item_id':X_test_arr[1, :], 'rating': y_test})

In [245]:
precisions = precision_at_k(model, predict_from_latent, train_ratings, test_ratings, no_users, no_items, k=10)

100%|██████████| 200/200 [00:11<00:00, 17.79it/s]


In [246]:
np.mean(precisions), np.std(precisions)

(0.004999999999999999, 0.02640496586292477)

In [247]:
(test_ratings.groupby('user_id').count())['item_id'].mean()

969.175

In [162]:
np.array(test_pids)[top_k[:10]]

array([ 60493,   3390, 155070, 217176, 236222,  84621,  55342,  71947,
        61432, 109592], dtype=int32)

In [248]:
### EXERCISE CELL ###
def recall_at_k(model, pred_func, train_ratings, test_ratings, no_users, no_items, k=5):
    recalls = []
    for user_id, user_test_rating in tqdm(test_ratings.groupby('user_id')):
        # Retrieve already-seen items
        train_pids = train_ratings[train_ratings['user_id'] == user_id]['item_id'].values
        # Retrieve the unseen items
        user_df = test_ratings[(test_ratings['user_id'] == user_id)]
        test_pids = user_df['item_id'].values
        relevant_pids = set(user_df.loc[user_df['rating'] > 2.5, 'item_id'].values)
        if len(relevant_pids) == 0:
          continue
        # Create testing dataframe
        test_df = pd.DataFrame({'user_id': user_id, 'item_id': test_pids})
        test_set = [test_df.user_id, test_df.item_id]
        # Make rating predictions for all items for that user
        predictions = model.predict(test_set, batch_size=2048)
        # predictions = pred_func(model, user_id, pid_array, train_ratings)
        predictions = predictions.reshape(-1)
        # Sort the items and het the top k
        top_k = np.argsort(predictions)[:k]
        top_k_items = set(np.array(test_pids)[top_k[:10]])
        recalls.append(len(top_k_items & relevant_pids) / len(relevant_pids))
    return recalls

In [249]:
recalls = recall_at_k(model, predict_from_latent, train_ratings, test_ratings, no_users, no_items, k=10)

100%|██████████| 200/200 [00:11<00:00, 17.74it/s]


In [250]:
np.mean(recalls), np.std(recalls)

(0.007417929292929293, 0.05500871034129213)

In [ ]:
### EXERCISE CELL ###
def map_at_k(model, pred_func, train_ratings, test_ratings, no_users, no_items, k=10):
    pid_array = np.arange(no_items, dtype=np.int32)
    maps = []
    for user_id, user_test_rating in tqdm(test_ratings.groupby('user_id')):
        train_pids = train_ratings[train_ratings['user_id'] == user_id]['item_id'].values
        test_pids = set(user_test_rating['item_id'].values)
        test_set = pd.DataFrame({'user_id': user_id, 'item_id': pid_array})
        test_set = [test_set.user_id, test_set.item_id]
        # Make rating predictions for all items for that user
        predictions = model.predict(test_set, batch_size=2048)
        predictions = predictions.reshape(-1)
        predictions[train_pids] = - math.inf
        partial_maps = []
        top_k = list(np.argsort(-predictions)[:k])
        for rank, item_id in enumerate(top_k):
            if item_id in test_pids:
                partial_maps.append(len(set(top_k[:rank+1]) & test_pids) / float(rank+1))
        maps.append(.0 if len(partial_maps) == 0 else np.sum(partial_maps) / float(k))
    return maps

In [ ]:
maps = map_at_k(model, predict_from_latent, train_ratings, test_ratings, no_users, no_items, k=10)

100%|██████████| 200/200 [01:07<00:00,  2.96it/s]


In [ ]:
np.mean(maps), np.std(maps)

(8.333333333333333e-05, 0.0011755613316388237)